<a href="https://colab.research.google.com/github/tttequila/Kaggle_20Q/blob/main/7B_MultiCoT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configuring your kaggle token, see more details in the [**Configure your API key**](https://ai.google.dev/gemma/docs/setup) section

### set up env

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%bash
mkdir ~/.kaggle
# change the first path to your path of kaggle.json
cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
%%bash
pip install -q -U torch immutabledict sentencepiece

In [4]:
!git clone https://github.com/google/gemma_pytorch.git

fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.


### set up Gemma lib

In [5]:
# login the kaggle (need to store you kaggle.json to your google dirve)
import kagglehub
kagglehub.login()

import sys
sys.path.append("gemma_pytorch/gemma")
sys.path.append("gemma_pytorch")
import contextlib, torch


In [6]:
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM, GemmaModel

In [7]:
import torch
import gemma
import itertools
from typing import Iterable
from typing import Any, List, Optional, Sequence, Tuple, Union
import os
import re

In [8]:
@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)


def interleave_unequal(x, y):
    '''
        Interleave two lists of unequal length.
    '''
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]

### Building Agents

<details>
  <summary> model </summary>
  
  - `self.forward()`: getting next token and corresponding logits
  - `self.generate()`: see summary below. May need to be rewriten if we wanna get the cumulative logits for the whole sentence  


</details>



<details>
  <summary> model.generate() </summary>
  
  - **prompts** | `Union[str, Sequence[str]]`: Your prompts
  - **device** | `Any`: Devices
  - **output_len** | `int`: max output length
  - **temperature** | `Union[float, None]`: temperature degree, controlling how variant its response could be  
  - **top_p** | `float`:
  - **top_k** | `int`:

  regarding temperature, top_p and top_k, check this [link](https://blog.csdn.net/REfusing/article/details/137866583)

</details>

#### Define Formatter


In [23]:
from typing import Iterable
import itertools

class PromptFormatter:

    '''
        formatter class to format the prompt text for the model.
        A general idea is
    '''

    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._template_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._template_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self._all_prompt = ''
        self.reset()

    def __repr__(self):
        return self._all_prompt

    def reset(self):
        self._all_prompt = ''
        # if system prompt is provided, add it to the prompt
        if self._system_prompt:
            self._all_prompt += self._template_user.format(self._system_prompt)
        # same for few shot examples
        if self._few_shot_examples:
            self.add_rounds(self._few_shot_examples, start_agent='user')

    def add_user_round(self, user_prompt: str):
        # add user round to the prompt
        self._all_prompt += self._template_user.format(user_prompt)

    def add_agent_round(self, model_response: str):
        self._all_prompt += self._template_model.format(model_response)

    def add_rounds(self, rounds: Iterable, start_agent: str):
        '''
            Apply a sequence of rounds to the formatter, starting with the specified agent.
        '''
        formatters = [self.add_agent_round, self.add_user_round] if start_agent == 'model' else [self.add_user_round, self.add_agent_round] # here, self.model and self.user are functions definded above
        formatters = itertools.cycle(formatters)
        for fmt, round in zip(formatters, rounds):
            fmt(round)
        return self

    # def add_end_token(self):
    #     self._all_prompt += f"{self._end_token}\n"

    def add_new_round(self, player: str, prompt:str = None, end_token: bool = False):
        self._all_prompt += f"{self._start_token}{player}\n"
        if prompt:
            self._all_prompt += f'{prompt}'
        if end_token:
            # self.add_end_token()
            self._all_prompt += f"{self._end_token}\n"

    def formate_MCQA(self):
        raise NotImplementedError

sys_prompt = 'You are a highly knowledgeable naturalist with extensive knowledge about objects, places, and people around the world, almost like a search engine. You also possess strong reasoning abilities, allowing you to deduce the answer to your query using existing knowledge and information. \
You need to utilize your knowledge and reasoning skills to play a game of 20 questions. \
In each round of the game, choose the binary attribute most likely to aid your deduction, such as "a cross on the flag." \
Using this attribute, ask a question in English. Generally, the most helpful attribute provides the maximum information gain, meaning it significantly reduces the entropy of your guess, regardless of whether the answer is **yes** or **no**. For example, if you believe asking about the color is most helpful for your deduction, you should ask, "Is it a thing with a red color?" \
And after your thorough thinking, you need to wrap some key point with double stars and their corresponding content with double quotation marks (such as **key point**: "corresponding content").  Here are few examples of how to use Chain of thought:'

few_shot_examples = [

'Now give me an example of thinking with the Chain of Thought',

# 'Given Information: ["is a country", "is in Europe", "its flag is with red color"] \
# Considerations: \
# 1. Geographical Location: European countries can be divided into regions such as Northern Europe, Southern Europe, Western Europe, and Eastern Europe. This division can effectively narrow down the options. \
# 2. Population and Area: Considering population and area can also eliminate some countries. \
# 3. Other Colors on the Flag: We know the flag has red; we can further narrow down by asking about other colors. \
# 4. Language: European countries have various languages; asking about the official language can narrow it further. \
# Reasoning: Southern Europe includes a few countries such as Spain, Italy, Portugal, Greece, etc. Determining if it is in Southern Europe can immediately eliminate about three-quarters of European countries. \
# Compared to questions about flag colors or language, geographical location questions are more evenly distributed, and a yes or no answer will significantly narrow down the range. \
# Possible Next Steps: \
# If the answer is yes, we can further inquire about other notable characteristics of these countries, such as language or history. \
# If the answer is no, we can turn our attention to Western, Eastern, or Northern Europe and continue narrowing down the options.\
# **Key Attribute** "in Southern Europe"\
# **Question** "Is this country in Southern Europe?"',

# 'Now let us play a game of 20 questions. You will be the questioner, and I will be the answerer.',

# 'It is the first round, so i will randomly choose a attribute from ["country", "city", "landmark"] as the first attribut. Here I choose country as the first attribute.\
# **Key Attribute** "a country"\
# **Question** "Is it a country?"',

# '**no**',

# 'Given information: ["It is not a country", "It is either a landmark or a city"] \
# Considerations: \
# 1. Type of Entity: Determining whether it is a city or a landmark can greatly narrow down our options. \
# 2. Geographical Location: If we can determine the continent, we can significantly reduce the number of possible cities or landmarks. \
# 3. Cultural/Historical Significance: Asking about cultural or historical significance can also be helpful, but it might be too broad at this stage. \
# Reasoning: To effectively narrow it down, the most efficient next step is to determine if it is a city or a landmark. By determining if it is a city, we can immediately focus on known cities globally, which are fewer and more well-documented compared to all possible landmarks. \
# If it is not a city, we can then focus on landmarks, which might require more specific questions but will also significantly narrow our scope. \
# Possible Next Steps: \
# If the answer is yes, we can then ask about the continent or specific features of the city. \
# If the answer is no, we can ask about the type of landmark or its location. \
# **Key Attribute** "a city\
# **Question** "Is it a city?"',

# '**no**',

# 'Given information: ["It is not a country", "It is either a landmark or a city", "It is not a landmark"] \
# Considerations: \
# Alright, so it is not a city. Now we know it must be a landmark. Given this, our goal is to identify the type or location of the landmark. \
# 1. Type of Landmark: There are various types such as natural landmarks (mountains, rivers), man-made landmarks (statues, buildings), or historical sites. \
# 2. Geographical Location: Identifying the continent or region can greatly narrow down the possibilities. \
# To get a high information gain, let us narrow down the type of landmark first. \
# Reasoning: Landmarks can be broadly categorized into man-made or natural. This binary distinction will help us immediately focus on a more specific set of possibilities. \
# Man-made landmarks include famous buildings, statues, monuments, etc., while natural landmarks include mountains, rivers, and natural formations. \
# Possible Next Steps: \
# If the answer is yes, we can then ask about the purpose or era of the man-made structure. \
# If the answer is no, we can focus on identifying the type of natural landmark or its location. \
# **Key Attribute** "a man-made landmark"\
# **Question** "Is it a man-made landmark?"',

# '**no**',

'Given information: ["It is not a country", "It is either a landmark or a city", "It is not a landmark", "It is not a man-made landmark"] \
Considerations: \
1. Type of Natural Landmark: Common natural landmarks include mountains, rivers, lakes, forests, deserts, etc. \
2. Geographical Location: Identifying the continent or region can still be very helpful. \
To further narrow down, let us focus on the type of natural landmark. \
Reasoning: River are one of the most well-known types of natural landmarks. \
Possible Next Steps: \
If the answer is yes, we can then focus on identifying which river by asking about its specific features or location. \
If the answer is no, we can eliminate a significant category and focus on other types of natural landmarks like mountains, lakes, deserts, or forests. \
**Key Attribute** "a river"\
**Question** "Is it a river?"',

'**yes**',

'Given information: ["It is not a country", "It is either a landmark or a city", "It is not a landmark", "It is a river"] \
Considerations: \
Great, so it is a river. Now, we need to narrow down which river it could be. To do this effectively, identifying the continent or region will significantly help. \
1. Geographical Location: Identifying the continent or region can narrow down the list of possible rivers considerably. \
2. Length or Size: Asking about whether it is among the longest rivers might also help. \
3. Famous Rivers: There are some rivers that are globally well-known due to their significance. \
To achieve a high information gain, let us focus on the geographical location. \
Reasoning: Asia has some of the world us most significant and longest rivers, such as the Yangtze, Ganges, and Mekong. \
Possible Next Steps: \
If the answer is yes, we can focus on Asian rivers specifically. \
If the answer is no, we can eliminate Asia and consider other continents like Africa, Europe, South America, or North America. \
**Key Attribute** "located in Asia"\
**Question** "Is this reiver located in Asia?"',

'**yes**',

'Given information: ["It is not a country", "It is either a landmark or a city", "It is not a landmark", "It is a river", "It is located in Asia"] \
Excellent, the river is located in Asia. This narrows down our options significantly. Now, we need to focus on identifying which river it could be. \
Considerations: \
1. Famous Asian Rivers: Asia has several famous rivers, such as the Yangtze, Ganges, Mekong, Indus, and Yellow River. \
2. Geographical Features: We could ask about the river\'s length, the countries it flows through, or any significant historical or cultural associations. \
3. To narrow it down further, let us focus on whether it is one of the most well-known rivers. \
Reasoning: These three rivers are among the longest and most significant in Asia. \
Possible Next Steps: \
If the answer is yes, we can focus on distinguishing between these three. \
If the answer is no, we can focus on other significant rivers like the Ganges, Indus, or Brahmaputra. \
**Key Attribute** "one of the three longest rivers in Asia"\
**Question** "Is this river one of the three longest rivers in Asia (Yangtze, Yellow, or Mekong)?"',

'**yes**',

'Given information: ["It is not a country", "It is either a landmark or a city", "It is not a landmark", "It is a river", "It is located in Asia", "It is one of the three longest rivers in Asia"] \
Great, we now know it is one of the three longest rivers in Asia: the Yangtze, Yellow, or Mekong. Let us narrow it down further by focusing on specific characteristics of these rivers. \
Considerations: \
1. Geographical Location: We can ask about the countries the river flows through. \
2. Cultural/Historical Significance: Each of these rivers has distinct cultural and historical significance. \
To narrow it down further, let us ask about the country. \
Reasoning: Both the Yangtze and Yellow rivers flow through China, while the Mekong flows through several countries including China, but predominantly in Southeast Asia. \
Possible Next Steps: \
If the answer is yes, it will most likely be the Yangtze or Yellow River. \
If the answer is no, it must be the Mekong River. \
**Key Attribute** "flow through China"\
**Question** "Does this river flow through China?"',

'**yes**',

'Given information: ["It is not a country", "It is either a landmark or a city", "It is not a landmark", "It is a river", "It is located in Asia", "It is one of the three longest rivers in Asia", "It flows through China"] \
Great, so the river flows through China. This means it is either the Yangtze River or the Yellow River. To determine which one it is, we should focus on distinguishing features. \
Considerations: \
1. Length: The Yangtze is the longest river in Asia and the third-longest in the world. \
2. Color/Name: The Yellow River is named for its yellowish color due to loess sediments. \
Reasoning: The Yangtze River is the longest river in Asia. \
Possible Next Steps: \
If the answer is yes, it is the Yangtze River. \
If the answer is no, it must be the Yellow River. \
**Key Attribute** "the longest river in Asia"\
**Question** "Is this river the longest river in Asia?"',

'**yes**, Now guess the keyword. Give your analysis with chain of thought and known information. And surround your guess with double asterisks (*) in the end, such as **Guess** "your guess"',

'Given information: ["It is not a country", "It is either a landmark or a city", "It is not a landmark", "It is a river", "It is located in Asia", "It is one of the three longest rivers in Asia", "It flows through China", "It is the longest river in Asia"] \
Given the information provided, let us analyze and guess the keyword step by step: \
It is not a country: This eliminates any possibility of a country being the answer. \
It is either a landmark or a city: This narrows the options to two categories. \
It is not a landmark: This further narrows the options, confirming that the answer must be a city.\
It is a river: This information conflicts with the previous determination that it is a city unless miscommunication exists. Assuming this is accurate, we will consider this entity as a river instead of a city. \
It is located in Asia: This geographic specification helps limit the river to one continent. \
It is one of the three longest rivers in Asia: This is a critical clue that significantly narrows the options to three major rivers: the Yangtze, the Yellow River, and the Mekong. \
Since the provided information describes it as one of the three longest rivers in Asia, and we have confirmed it must be a river, not a city or landmark, we can focus on these three rivers. \
Final Step: \
Given the specified criteria, the keyword you are referring to could be either the Yangtze, Yellow River, or Mekong. However, you mentioned it is specifically a river and not a landmark or city, which leads directly to these rivers. \
For a direct guess based on the longest rivers in Asia, we could guess the Yangtze River since it is the longest river in Asia and fits all the criteria provided. \
Conclusion: The keyword is likely "Yangtze River", considering it is one of the longest and fits the location and other descriptions provided. \
**Guess**: "Yangtze River"',

'Correct!']

print(str(PromptFormatter(sys_prompt, few_shot_examples)))

<start_of_turn>user
You are a highly knowledgeable naturalist with extensive knowledge about objects, places, and people around the world, almost like a search engine. You also possess strong reasoning abilities, allowing you to deduce the answer to your query using existing knowledge and information. You need to utilize your knowledge and reasoning skills to play a game of 20 questions. In each round of the game, choose the binary attribute most likely to aid your deduction, such as "a cross on the flag." Using this attribute, ask a question in English. Generally, the most helpful attribute provides the maximum information gain, meaning it significantly reduces the entropy of your guess, regardless of whether the answer is **yes** or **no**. For example, if you believe asking about the color is most helpful for your deduction, you should ask, "Is it a thing with a red color?" And after your thorough thinking, you need to wrap some key point with double stars and their corresponding co

#### Define Agent

In [10]:
from ast import parse
import random

class GemmaAgent_Guesser:

    def __init__(self, model_variant, device='cuda:0', env="kaggle", output_len=200, system_prompt=None, few_shot_examples=None):
        # model initialization
        self.device = device
        self.model_variant = model_variant

        WEIGHTS_PATH = self._set_up_env(env)

        # Ensure that the tokenizer is present
        tokenizer_path = os.path.join(WEIGHTS_PATH, 'tokenizer.model')
        assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

        # Ensure that the checkpoint is present
        ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{model_variant}.ckpt')
        assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

        # loading model configuration
        model_config = get_config_for_2b() if "2b" in model_variant else get_config_for_7b()
        model_config.quant = "quant" in model_variant
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")

        with _set_default_tensor_type(model_config.get_dtype()):
            self.model = GemmaForCausalLM(model_config)
            self.model.load_weights(ckpt_path)
            self.model = self.model.to(self.device).eval()

        # agent args
        self.formatter = PromptFormatter(system_prompt=system_prompt,
                                         few_shot_examples=few_shot_examples)
        self.round_num = 0
        self.known_info = []
        self.last_key_attribute = None
        self.last_guess = None
        self.output_len = output_len

    def _set_up_env(self, env):

        if env == 'kaggle':
            print("Loading model in Kaggle, model weights will be searched within local directories.")

            # kaggle configuration
            KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
            if os.path.exists(KAGGLE_AGENT_PATH):
                WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, f"gemma/pytorch/{self.model_variant}/2")
            else:
                WEIGHTS_PATH = f"/kaggle/input/gemma/pytorch/{self.model_variant}/2"

        elif env == 'colab':
            print("Loading model in Colab, starting from downloading the model weights.")

            WEIGHTS_PATH = kagglehub.model_download(f'google/gemma/pyTorch/{self.model_variant}')
        else:
            raise ValueError("Argument 'env' should be in ['kaggle', 'colab']")

        return WEIGHTS_PATH

    def _parse_response(self, response : str):
        '''
            parse the response into a dictionary.
            may contain three keys: key_attribute, question, guess and their value respectively.
            e.g.: {"key_attribute": 'a country', "question": 'Is it a country?'} for a parsed asker response.
        '''
        pattern = re.compile(r'\*\*([^*]+)\*\*[^*"]+"([^"]+)"')
        matches = pattern.findall(response.lower())
        parse_dict = {'key attribute':None, 'question':None, 'guess':None}
        for k, v in matches:
          if 'attr' in k:
            parse_dict['key attribute'] = v
          elif 'ques' in k:
            parse_dict['question'] = v
          elif 'guess' in k:
            parse_dict['guess'] = v

        return parse_dict

    def _format_prompt(self, obs):
        # reset formatter
        self.formatter.reset()
        self.formatter.add_new_round('user', 'Remember how to use CoT as above, now let us play a new game from the begining.', True)
        # add played rounds
        rounds = interleave_unequal(obs["questions"], obs["answers"])
        self.formatter.add_rounds(rounds, start_agent='user')
        if obs["turnType"] == 'ask':
            self.formatter.add_new_round('user',
                                         'Now it is your turn to ask a question. Please using chain of thought to construct a new question based on what you have known and your knowledge storage. Do not forget to indicate key attribute, question with double asterisk (**?**) and their content with double quotation markers ("?") when it is needed, such as **Key Attribute** "your attribute" and **Question** "your question"',
                                         True)
        elif obs["turnType"] == 'guess':
            self.formatter.add_new_round('user',
                                         'Now guess the keyword. Give your analysis with chain of thought and known information. And surround a pointer of guess with double asterisks (**?**) and your guessed keyword with doublr quotation markers ("?") in the end, such as **Guess** "your guess"',
                                         True)
        # start of model response
        self.formatter.add_new_round('model', f'Given information: {str(self.known_info)}', False)

    def __call__(self, obs, cfg=None):
        '''
            Main function to interact with the model.
                1. update known information based on the observation and round number
                2. format the prompt with the observation
                3. generate response from the model
                4. parse the response and update information if needed
        '''
        print(f"=========== round: {self.round_num} ===========")
        self.round_num += 1
        # Update known information
        if self.round_num - 1 == 0:
            # for the first round, randomly choose an attribute as initialization
            initial_categories = ['person', 'thing', 'place']
            self.last_key_attribute = random.choice(initial_categories)
            self.else_categories = list(set(initial_categories) - set([self.last_key_attribute]))
            response = f'Is it a {self.last_key_attribute}?'
            round
            return response
        elif self.round_num - 1 == 1:
            # if it is the second round, grab the answer and update known information
            last_answer = obs["answers"][-1]
            if last_answer == 'yes':
                self.known_info.append(f'a {self.last_key_attribute}')
            else:
                self.known_info.append(f'not a {self.last_key_attribute}')
                self.known_info.append(f'either a {self.else_categories[0]} or a {self.else_categories[1]}')
        else:
            # other rounds, update known information
            last_answer = obs['answers'][-1]
            if last_answer == 'yes':
                self.known_info.append(f'is {self.last_key_attribute}')
            else:
                self.known_info.append(f'is not {self.last_key_attribute}')

        # # Update guessed keywords
        # self.known_info.append(f'keyword is not {self.last_guess}')
        # self.known_info = list(set(self.known_info))

        print(f"\nkey attribute: {self.last_key_attribute}, guess: {self.last_guess}")

        # Formatting prompt with observations
        self._format_prompt(obs)
        prompt = str(self.formatter)
        print(f"\nPrompt: {prompt}")
        # Getting response from LLM
        response = self.model.generate(prompt, device=self.device, output_len=self.output_len)
        print(f'\nTurn Type: {obs["turnType"]}\nResponse: {response}')

        # parse response and update information if needed
        parse_dict = self._parse_response(response)
        print(f'\nParse_dict: {parse_dict}')

        # if in an ask turn, try to grab the key attribute from the response and update last key attribute for the next information updating
        if obs["turnType"] == 'ask':
            # successfully grab the key attribute from the response
            if parse_dict['key attribute']:
                self.last_key_attribute = parse_dict['key attribute']
            # no key attribute parsed from the response, directly use question as key attribute
            elif parse_dict['question']:
                self.last_key_attribute  = parse_dict['question']
                ret = parse_dict['question']
                return ret
            # worst case, there is neither key attribute nor question parsed from the response, randomly choose an attribute
            else:
                random_alphabet = random.choice('abcdefghijklmnopqrstuvwxyz')
                ret = f'is it started with alphabet {random_alphabet}?'
                self.last_key_attribute = f'started with alphabet {random_alphabet}'
                return ret

            # parsed response has question, use question as response
            if parse_dict['question']:
                ret = parse_dict['question']
            # parsed response has no question but have key attribute, use key attribute as question
            else:

                ret = f'is it {self.last_key_attribute}?'
        # if in a guess turn, try to grab the guess from the response and update last guess for the next information updating
        elif obs["turnType"] == 'guess':
            if parse_dict['guess']:
                self.last_guess = parse_dict['guess']
                ret = parse_dict['guess']
            # if there is no guess parsed from the response, return empty string
            else:
                ret = ''
        else:
            raise ValueError('Invalid turnType.')

        return ret


### Debug

In [11]:
!pip -q install kaggle_environments
import kaggle_environments

In [12]:
# dummy agent
def simple_agent1(obs, cfg):
    # if agent is guesser and turnType is "ask"
    if obs['turnType'] == "ask": response = "Is it a duck?"
    elif obs.turnType == "guess": response = "duck"
    elif obs.turnType == "answer": response = "no"
    return response

def simple_agent2(obs, cfg):
    # if agent is guesser and turnType is "ask"
    if obs.turnType == "ask": response = "Is it a bird?"
    elif obs.turnType == "guess": response = "bird"
    elif obs.turnType == "answer": response = "no"
    return response

def simple_agent3(obs, cfg):
    # if agent is guesser and turnType is "ask"
    if obs.turnType == "ask": response = "Is it a pig?"
    elif obs.turnType == "guess": response = "pig"
    elif obs.turnType == "answer": response = "no"
    return response

def simple_agent4(obs, cfg):
    # if agent is guesser and turnType is "ask"
    if obs.turnType == "ask": response = "Is it a cow?"
    elif obs.turnType == "guess": response = "cow"
    elif obs.turnType == "answer": response = "no"
    return response

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
# **IMPORTANT:** Define agent as a global so you only have to load
# the agent you need. Loading both will likely lead to OOM.
# del agent
agent = None
print(agent)

# def get_agent(name: str):
#     global agent

#     if agent is None and name == 'questioner':
#         agent = GemmaAgent_Guesser(VARIANT, MACHINE_TYPE, 'colab',
#                                    sys_prompt, few_shot_examples)
#     elif agent is None and name == 'answerer':
#         raise NotImplementedError
#     assert agent is not None, "Agent not initialized."

#     return agent


# def agent_fn(obs, cfg):
#     if obs.turnType == "ask":
#         response = get_agent('questioner')(obs)
#     elif obs.turnType == "guess":
#         response = get_agent('questioner')(obs)
#     elif obs.turnType == "answer":
#         response = get_agent('answerer')(obs)
#     if response is None or len(response) <= 1:
#         return "yes"
#     else:
#         return response


None


In [14]:
debug_config = {'episodeSteps': 10,     # initial step plus 3 steps per round (ask/answer/guess)
                'actTimeout': 5,       # agent time per round in seconds; default is 60
                'runTimeout': 60,      # max time for the episode in seconds; default is 1200
                'agentTimeout': 3600}  # obsolete field; default is 3600
env = kaggle_environments.make("llm_20_questions", debug_config, debug=True)
print(f'the key word is: {kaggle_environments.envs.llm_20_questions.llm_20_questions.keyword}\nwith alts as {kaggle_environments.envs.llm_20_questions.llm_20_questions.alts}')

the key word is: Oak table
with alts as []


In [15]:
import time
# Choose variant and machine type
VARIANT = '7b-it-quant'
MACHINE_TYPE = 'cuda'
agent = None


In [24]:
# agent = GemmaAgent_Guesser(VARIANT, MACHINE_TYPE, 'colab', 150, sys_prompt, few_shot_examples)
agent.formatter._sys_prompt = sys_prompt
agent.formatter._few_shot_examples = few_shot_examples
agent.round_num = 4
agent.last_key_attribute = 'a person'
agent.last_guess = 'pig'
agent.known_info = ['not a place', 'either a person or a thing', 'not a thing']
# print(str(agent.formatter))
obs = {'remainingOverageTime': 300, 'questions': ['Is it a place?', 'Is it a thing?', 'Is it a person?'], 'guesses': ['pig', 'pig', 'pig'], 'answers': ['no', 'no', 'yes'], 'role': 'guesser', 'turnType': 'ask', 'keyword': '', 'category': ''}
start = time.time()
agent(obs)
print(f"TIME: {(time.time()-start):.4f}s")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


=========== round: 4 ===========

key attribute: a person, guess: pig

Prompt: <start_of_turn>user
You are a highly knowledgeable naturalist with extensive knowledge about objects, places, and people around the world, almost like a search engine. You also possess strong reasoning abilities, allowing you to deduce the answer to your query using existing knowledge and information. You need to utilize your knowledge and reasoning skills to play a game of 20 questions. In each round of the game, choose the binary attribute most likely to aid your deduction, such as "a cross on the flag." Using this attribute, ask a question in English. Generally, the most helpful attribute provides the maximum information gain, meaning it significantly reduces the entropy of your guess, regardless of whether the answer is **yes** or **no**. For example, if you believe asking about the color is most helpful for your deduction, you should ask, "Is it a thing with a red color?" And after your thorough thinkin

In [17]:
game_output = env.run(agents=[simple_agent1,simple_agent2, simple_agent3, simple_agent4])
print(game_output[9])

[{'action': 'duck', 'reward': 0, 'info': {}, 'observation': {'remainingOverageTime': 300, 'step': 9, 'questions': ['Is it a duck?', 'Is it a duck?', 'Is it a duck?'], 'guesses': ['duck', 'duck', 'duck'], 'answers': ['no', 'no', 'no'], 'role': 'guesser', 'turnType': 'ask', 'keyword': '', 'category': ''}, 'status': 'DONE'}, {'action': '', 'reward': 0, 'info': {}, 'observation': {'remainingOverageTime': 300, 'questions': ['Is it a duck?', 'Is it a duck?', 'Is it a duck?'], 'guesses': ['duck', 'duck', 'duck'], 'answers': ['no', 'no', 'no'], 'role': 'answerer', 'turnType': 'answer', 'keyword': 'Oak table', 'category': 'things'}, 'status': 'DONE'}, {'action': 'pig', 'reward': 0, 'info': {}, 'observation': {'remainingOverageTime': 300, 'questions': ['Is it a pig?', 'Is it a pig?', 'Is it a pig?'], 'guesses': ['pig', 'pig', 'pig'], 'answers': ['no', 'no', 'no'], 'role': 'guesser', 'turnType': 'ask', 'keyword': '', 'category': ''}, 'status': 'DONE'}, {'action': '', 'reward': 0, 'info': {}, 'obs

In [18]:
# env.render(mode="ipython", width=600, height=500)